In [ ]:
# Lab 4 - part2
### Edge Detection

https://github.com/scikit-image/scikit-image/blob/master/skimage/filters/edges.py

http://scikit-image.org/docs/0.11.x/auto_examples/plot_edge_filter.html

http://scikit-image.org/docs/dev/auto_examples/edges/plot_canny.html



In [ ]:
from commonfunctions import *
from skimage import filters
import matplotlib.pyplot as plt
from skimage import feature
from skimage.filters import gaussian
from scipy.signal import convolve2d
from skimage.exposure import rescale_intensity
%matplotlib inline
%load_ext autoreload
%autoreload 2


In [ ]:
'''
1. Built-in Sobel, Prewill, Roberts
2. Custom Sobel (V-H-Both)
3. Custom LoG
'''

In [ ]:
# 1. Built-in Sobel, Prewill, Roberts


img = rgb2gray(io.imread('circuit.tif'))
edge_roberts = filters.roberts(img)
edge_sobel = filters.sobel(img)
edge_sobel_h = filters.sobel_h(img)
edge_sobel_v = filters.sobel_v(img)
edge_prewitt = filters.prewitt(img)
edge_canny = feature.canny(img, sigma=1)

show_images([img, edge_roberts], ["Original Image", "Robert Edge Detection"])
show_images([img, edge_sobel_v], ["Original Image", "Sobel_v Edge Detection"])
show_images([img, edge_sobel_h], ["Original Image", "Sobel_h Edge Detection"])
show_images([img, edge_sobel], ["Original Image", "Sobel Edge Detection"])
show_images([img, edge_prewitt], ["Original Image", "Prewitt Edge Detection"])
show_images([img, edge_canny], ["Original Image", "Canny Edge Detection"])



In [ ]:
# 2. Custom Sobel (V-H-Both)

from skimage.exposure import rescale_intensity

def custom_sobel(img, thresh):
    hy = [[-1,0,1],
          [-2,0,2],
          [-1,0,1]]
    hx = [[-1,-2,-1],
          [0,0,0],
          [1,2,1]]
    xComp = convolve2d(img, hx)
    yComp = convolve2d(img, hy)
    
    edgeStrength = np.sqrt(xComp ** 2 + yComp ** 2)

    
    before_thresh = edgeStrength.copy()
    for i in range(edgeStrength.shape[0]):
        for j in range(edgeStrength.shape[1]):
            if edgeStrength[i][j] > thresh:
                edgeStrength[i][j] = 255
            else:
                edgeStrength[i][j] = 0
    
    show_images([img, xComp, yComp, before_thresh, edgeStrength], ["Original", "X component","Y component", "before_thresh", "edgeStrength"])
    
    
img = rgb2gray(io.imread("circuit.tif"))
custom_sobel(img, 128)

In [ ]:
# 3. Custom LoG
def apply_filter(img, f, sigma, thres):
    img_g = gaussian(img, sigma) # apply the gaussian filter
    
    img_g_f = convolve2d(img_g, f, boundary="symm") # apply the f filter
    img_g_f_flat = img_g_f.flatten() # flatten
    img_g_f_flat[[i for i in range(len(img_g_f_flat)) if img_g_f_flat[i] < 0]] = 0; # clip
    img_g_f_flat[[i for i in range(len(img_g_f_flat)) if img_g_f_flat[i] > 1]] = 1; # clip
    img_g_f = np.reshape(img_g_f_flat, img_g_f.shape)

    img_g_f_flat = rescale_intensity(img_g_f_flat, in_range=(0, 1.0), out_range=(0, 255)) # rescale
    img_g_f_flat = img_g_f_flat.astype(np.uint8) # cast
    img_g_f_flat[[i for i in range(len(img_g_f_flat)) if img_g_f_flat[i] <= thres]] = 0 # apply threshold
    img_g_f_flat[[i for i in range(len(img_g_f_flat)) if img_g_f_flat[i] > thres]] = 255 # apply threshold
    img_g_f_th = np.reshape(img_g_f_flat, img_g_f.shape)
    
    return (img_g, img_g_f, img_g_f_th)

thres = 2
sigma = 2.8

f1 = np.array([
    [-1, -1, -1]  
    ,[-1, 8, -1]   
    ,[-1, -1, -1] 
])

f2 = np.array([
    [0, 1, 0]
    ,[1, -4, 1] 
    ,[0, 1, 0]
])

img_g, img_g_f1, img_g_f1_th = apply_filter(img, f1, sigma, thres)
img_g, img_g_f2, img_g_f2_th = apply_filter(img, f2, sigma, thres)


show_images([img, img_g, img_g_f1, img_g_f1_th, img_g_f2, img_g_f2_th],
            ["img", "img_g", "img_g_f1", "img_g_f1_th", "img_g_f2", "img_g_f2_th"])



In [ ]:
'''
Optional
1-Make edge detection
2-Multiply by factor < 1
3-Subtract (Orginal Image) - (Edge Detection Image)
'''